## 10. Collaboration

### 87 Define a Root `Exception` to Insulate Callers from APIs

In [1]:
import logging

In [2]:
# my_module.py

def determine_weight(volume, density):
    if density <= 0:
        raise ValueError('Density must be positive')

try:
    determine_weight(1, 0)
except ValueError:
    pass
else:
    assert False

In [4]:
# my_module.py

class Error(Exception):
    """Base-class for all exceptions raised by this module."""

class InvalidDensityError(Error):
    """There was a problem with a provided density value."""

class InvalidVolumeError(Error):
    """There was a problem with the provided weight value."""

def determine_weight(volume, density):
    if density < 0:
        raise InvalidDensityError('Density must be positive')
    if volume < 0:
        raise InvalidVolumeError('Volume must be positive')
    if volume == 0:
        density / volume

In [5]:
# mimic module
class my_module:
    Error = Error
    InvalidDensityError = InvalidDensityError

    @staticmethod
    def determine_weight(volume, density):
        if density < 0:
            raise InvalidDensityError('Density must be positive')
        if volume < 0:
            raise InvalidVolumeError('Volume must be positive')
        if volume == 0:
            density / volume

try:
    weight = my_module.determine_weight(1, -1)
except my_module.Error:
    logging.exception('Unexpected error')
else:
    assert False

ERROR:root:Unexpected error
Traceback (most recent call last):
  File "<ipython-input-5-5f912faf51a8>", line 16, in <module>
    weight = my_module.determine_weight(1, -1)
  File "<ipython-input-5-5f912faf51a8>", line 9, in determine_weight
    raise InvalidDensityError('Density must be positive')
InvalidDensityError: Density must be positive


In [6]:
SENTINEL = object()
weight = SENTINEL
try:
    weight = my_module.determine_weight(-1, 1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error:
    logging.exception('Bug in the calling code')
else:
    assert False

assert weight is SENTINEL

ERROR:root:Bug in the calling code
Traceback (most recent call last):
  File "<ipython-input-6-24d29c07a718>", line 4, in <module>
    weight = my_module.determine_weight(-1, 1)
  File "<ipython-input-5-5f912faf51a8>", line 11, in determine_weight
    raise InvalidVolumeError('Volume must be positive')
InvalidVolumeError: Volume must be positive


In [7]:
try:
    weight = SENTINEL
    try:
        weight = my_module.determine_weight(0, 1)
    except my_module.InvalidDensityError:
        weight = 0
    except my_module.Error:
        logging.exception('Bug in the calling code')
    except Exception:
        logging.exception('Bug in the API code!')
        raise  # Re-raise exception to the caller
    else:
        assert False
    
    assert weight == 0
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Bug in the API code!
Traceback (most recent call last):
  File "<ipython-input-7-10a151fb0c7d>", line 4, in <module>
    weight = my_module.determine_weight(0, 1)
  File "<ipython-input-5-5f912faf51a8>", line 13, in determine_weight
    density / volume
ZeroDivisionError: division by zero
ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-7-10a151fb0c7d>", line 4, in <module>
    weight = my_module.determine_weight(0, 1)
  File "<ipython-input-5-5f912faf51a8>", line 13, in determine_weight
    density / volume
ZeroDivisionError: division by zero


In [8]:
# my_module.py

class NegativeDensityError(InvalidDensityError):
    """A provided density value was negative."""

def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError('Density must be positive')

In [9]:
try:
    my_module.NegativeDensityError = NegativeDensityError
    my_module.determine_weight = determine_weight
    try:
        weight = my_module.determine_weight(1, -1)
    except my_module.NegativeDensityError:
        raise ValueError('Must supply non-negative density')
    except my_module.InvalidDensityError:
        weight = 0
    except my_module.Error:
        logging.exception('Bug in the calling code')
    except Exception:
        logging.exception('Bug in the API code!')
        raise
    else:
        assert False
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-9-fc3450e1cddd>", line 5, in <module>
    weight = my_module.determine_weight(1, -1)
  File "<ipython-input-8-6bd870f51a07>", line 8, in determine_weight
    raise NegativeDensityError('Density must be positive')
NegativeDensityError: Density must be positive

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-9-fc3450e1cddd>", line 7, in <module>
    raise ValueError('Must supply non-negative density')
ValueError: Must supply non-negative density


In [10]:
# my_module.py

class Error(Exception):
    """Base-class for all exceptions raised by this module."""

class WeightError(Error):
    """Base-class for weight calculation errors."""

class VolumeError(Error):
    """Base-class for volume calculation errors."""

class DensityError(Error):
    """Base-class for density calculation errors."""

> - 모듈에서 사용할 최상위 예외를 정의하면 API 사용자들이 자신을 API로부터 보호할 수 있다.
> - 최상위 예외를 잡아내면 API를 소비하는 코드의 버그를 쉽게 찾을 수 있다.
> - 파이썬 `Exception` 기반 클래스를 잡아내면 API 구현의 버그를 쉽게 찾을 수 있다.
> - 중간 단계의 최상위 예외를 사용하면, 미래에 새로운 타입의 예외를 API에 추가할 때 API를 사용하는 코드가 깨지는 일을 방지할 수 있다.